<a href="https://colab.research.google.com/github/mbeirne11/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(['EIN','NAME'],axis = 1)

In [3]:
# Determine the number of unique values in each column.
application_df.value_counts()

APPLICATION_TYPE  AFFILIATION       CLASSIFICATION  USE_CASE      ORGANIZATION  STATUS  INCOME_AMT     SPECIAL_CONSIDERATIONS  ASK_AMT   IS_SUCCESSFUL
T3                Independent       C1000           Preservation  Trust         1       0              N                       5000      1                3507
                  CompanySponsored  C1000           Preservation  Trust         1       0              N                       5000      0                1379
                                    C1200           Preservation  Association   1       0              N                       5000      0                1122
                  Independent       C1000           Preservation  Trust         1       0              N                       5000      0                1102
                                    C1200           Preservation  Trust         1       0              N                       5000      1                1052
                                                      

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
application_df['APPLICATION_TYPE'].unique()

array(['T10', 'T3', 'T5', 'T7', 'T4', 'T6', 'T2', 'T9', 'T19', 'T8',
       'T13', 'T12', 'T29', 'T25', 'T14', 'T17', 'T15'], dtype=object)

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T17', 'T15', 'T29', 'T14', 'T25', 'T2','T12','T13','T9']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
count = application_df['CLASSIFICATION'].value_counts()

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
df = pd.DataFrame(count)

In [9]:
df = df.reset_index()

In [10]:
df

,index,CLASSIFICATION
0,C1000,17326
1,C2000,6074
2,C1200,4837
3,C3000,1918
4,C2100,1883
...,...,...
66,C4120,1
67,C8210,1
68,C2561,1
69,C4500,1


In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []
for classification in df.iterrows():
  if (classification[1]['CLASSIFICATION'] == 1):
    classifications_to_replace.append(classification[1]['index'])
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Other       26
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C1720        6
C2400        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [12]:
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [13]:
from sklearn.preprocessing import StandardScaler
# Scaling the numeric columns
scaler = StandardScaler()
scaler.fit(application_df[['ASK_AMT']])
transformed_data = scaler.transform(application_df[['ASK_AMT']])
# Creating a DataFrame with with the scaled data
transformed_data_df = pd.DataFrame(transformed_data)

# Display sample data
transformed_data_df

,0
0,-0.031725
1,-0.030536
2,-0.031725
3,-0.031706
4,-0.030146
...,...
34294,-0.031725
34295,-0.031725
34296,-0.031725
34297,-0.031725


In [14]:
scaled_df = application_df.copy()
scaled_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [15]:
scaled_df['ASK_AMT']=transformed_data_df[0]

In [16]:
scaled_df['ASK_AMT']

0       -0.031725
1       -0.030536
2       -0.031725
3       -0.031706
4       -0.030146
           ...   
34294   -0.031725
34295   -0.031725
34296   -0.031725
34297   -0.031725
34298    0.387138
Name: ASK_AMT, Length: 34299, dtype: float64

In [17]:
dummy_df = pd.get_dummies(scaled_df.drop(['IS_SUCCESSFUL','STATUS','ASK_AMT'],axis=1))

In [18]:
dummy_df['IS_SUCCESSFUL']=scaled_df['IS_SUCCESSFUL']
dummy_df['STATUS']=scaled_df['STATUS']
dummy_df['ASK_AMT']=scaled_df['ASK_AMT']
dummy_df

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,IS_SUCCESSFUL,STATUS,ASK_AMT
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,1,-0.031725
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,1,-0.030536
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,1,-0.031725
3,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,1,-0.031706
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,1,-0.030146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,-0.031725
34295,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,-0.031725
34296,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,-0.031725
34297,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,1,-0.031725


In [19]:
# Split our preprocessed data into our features and target arrays
y = dummy_df['IS_SUCCESSFUL']
X = dummy_df.drop('IS_SUCCESSFUL',axis=1)

In [20]:
X

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,-0.031725
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,-0.030536
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,1,-0.031725
3,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,-0.031706
4,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,-0.030146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,-0.031725
34295,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,-0.031725
34296,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,-0.031725
34297,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1,-0.031725


In [21]:
from sklearn.decomposition import PCA
# Create a PCA model
pca = PCA(n_components=3)

In [22]:
# Use the PCA model with `fit_transform` to reduce to
# three principal components.
pca_data = pca.fit_transform(dummy_df)

# View the first five rows of the DataFrame.
pca_data

array([[-0.08124148,  0.58396217,  1.6891265 ],
       [ 0.04276259, -0.58260516,  0.05294574],
       [-0.19979834,  1.93043624,  0.7441538 ],
       ...,
       [-0.13969581,  1.13974953, -0.85626992],
       [-0.10198453,  0.81074413,  1.563884  ],
       [ 0.4590226 , -0.44086769,  0.21748336]])

In [23]:
len(pca_data)

34299

In [24]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.24938282, 0.20436553, 0.10999716])

In [25]:
# Create a new DataFrame with the PCA data.
pca_dict = {}
for i in range(0,3):
  pca_dict[f'pca{i+1}']=pca_data[:,i]
# Display sample data
pca_df = pd.DataFrame(pca_dict)

In [26]:
pca_df

,pca1,pca2,pca3
0,-0.081241,0.583962,1.689126
1,0.042763,-0.582605,0.052946
2,-0.199798,1.930436,0.744154
3,-0.018194,-0.066291,-0.630879
4,0.089735,-0.929440,0.805570
...,...,...,...
34294,-0.093688,0.843828,1.500998
34295,-0.200976,1.941200,0.753314
34296,-0.139696,1.139750,-0.856270
34297,-0.101985,0.810744,1.563884


In [27]:
y

0        1
1        1
2        0
3        1
4        1
        ..
34294    0
34295    0
34296    0
34297    1
34298    0
Name: IS_SUCCESSFUL, Length: 34299, dtype: int64

## Compile, Train and Evaluate the Model

In [28]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(pca_df, y, random_state=78)

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=7, activation="relu", input_dim=3))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=7, activation="relu"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=7, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 28        
                                                                 
 dense_1 (Dense)             (None, 7)                 56        
                                                                 
 dense_2 (Dense)             (None, 7)                 56        
                                                                 
 dense_3 (Dense)             (None, 1)                 8         
                                                                 
Total params: 148 (592.00 Byte)
Trainable params: 148 (592.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 7s 5ms/step - loss: 0.5810 - accuracy: 0.7212
Epoch 2/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5346 - accuracy: 0.7363
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5234 - accuracy: 0.7428
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5138 - accuracy: 0.7446
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5029 - accuracy: 0.7490
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4923 - accuracy: 0.7624
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4824 - accuracy: 0.7658
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4708 - accuracy: 0.7757
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4595 - accuracy: 0.7890
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4494 - accura

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.1267 - accuracy: 0.9404 - 497ms/epoch - 2ms/step
Loss: 0.12669572234153748, Accuracy: 0.9404081702232361


In [33]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
